### Синтаксический анализ

In [1]:
with open('../data/sentences.txt') as f:
    text = f.read()
    
print(text)

Google с помощью искусственного интеллекта испек печенье .

Овечкин стал первой звездой дня в НХЛ .

На Кубани задержали подозреваемого в двойном убийстве . 

В Ростове суд вынес приговор банде «амазонок» .

Чиновники закрасили мемориал жертвам теракта в питерском метро .

Клуб «Наполи» продлил контракт с Гуламом .



Вызов SyntaxNet:

Обработка результатов парсинга:

In [2]:
! cat ../data/sentences.txt | docker run --rm -i inemo/syntaxnet_rus > data.conll

I syntaxnet/term_frequency_map.cc:103] Loaded 34 terms from ./syntaxnet/models/Russian-SynTagRus/label-map.
I syntaxnet/embedding_feature_extractor.cc:35] Features: stack.child(1).label stack.child(1).sibling(-1).label stack.child(-1).label stack.child(-1).sibling(1).label stack.child(2).label stack.child(-2).label stack(1).child(1).label stack(1).child(1).sibling(-1).label stack(1).child(-1).label stack(1).child(-1).sibling(1).label stack(1).child(2).label stack(1).child(-2).label; input.token.morphology-set input(1).token.morphology-set input(2).token.morphology-set input(3).token.morphology-set stack.token.morphology-set stack.child(1).token.morphology-set stack.child(1).sibling(-1).token.morphology-set stack.child(-1).token.morphology-set stack.child(-1).sibling(1).token.morphology-set stack.child(2).token.morphology-set stack.child(-2).token.morphology-set stack(1).token.morphology-set stack(1).child(1).token.morphology-set stack(1).child(1).sibling(-1).token.morphology-set stack(

SOV-тройки:

In [3]:
from nltk import DependencyGraph
import codecs

processed_sentences = []
sentence = []
for line in codecs.open('data.conll', 'r', 'utf-8'):
    if len(line) == 1:
        processed_sentences.append(sentence)
        sentence = []
    else:
        word = line.split("\t")
        sentence.append(word)

deps = []
for sentence in processed_sentences:
    s = u""
    for line in sentence:
        s += u"\t".join(line) + u'\n'
    deps.append(s)

for sent_dep in deps:
    graph = DependencyGraph(tree_str=sent_dep)
    for triple in graph.triples():
        if triple:
            print(triple)
    tree = graph.tree()
    print(tree.pretty_print())

(('испек', 'VERB'), 'dobj', ('Google', 'NOUN'))
(('испек', 'VERB'), 'nmod', ('помощью', 'NOUN'))
(('помощью', 'NOUN'), 'case', ('с', 'ADP'))
(('помощью', 'NOUN'), 'nmod', ('интеллекта', 'NOUN'))
(('интеллекта', 'NOUN'), 'amod', ('искусственного', 'ADJ'))
(('испек', 'VERB'), 'nsubj', ('печенье', 'NOUN'))
(('испек', 'VERB'), 'punct', ('.', 'PUNCT'))
               испек                           
   ______________|__________                    
  |       |      |       помощью               
  |       |      |     _____|__________         
  |       |      |    |            интеллекта  
  |       |      |    |                |        
Google печенье   .    с          искусственного

None
(('стал', 'VERB'), 'nsubj', ('Овечкин', 'NOUN'))
(('стал', 'VERB'), 'nmod', ('звездой', 'NOUN'))
(('звездой', 'NOUN'), 'amod', ('первой', 'ADJ'))
(('звездой', 'NOUN'), 'nmod', ('дня', 'NOUN'))
(('звездой', 'NOUN'), 'nmod', ('НХЛ', 'NOUN'))
(('НХЛ', 'NOUN'), 'case', ('в', 'ADP'))
(('стал', 'VERB'), 'punct

In [4]:
for sent_dep in deps:
    graph = DependencyGraph(tree_str=sent_dep)
    sov = {}
    for triple in graph.triples():
        if triple:
            if triple[0][1] == 'VERB':
                sov[triple[0][0]] = {'subj':'','obj':''}
    for triple in graph.triples():
        if triple:
            if triple[1] == 'nsubj':
                if triple[0][1] == 'VERB':
                    sov[triple[0][0]]['subj']  = triple[2][0]
            if triple[1] == 'dobj':
                if triple[0][1] == 'VERB':
                    sov[triple[0][0]]['obj'] = triple[2][0]

    for verb in sov:
        print(verb,sov[verb])

испек {'subj': 'печенье', 'obj': 'Google'}
стал {'subj': 'Овечкин', 'obj': ''}
задержали {'subj': '', 'obj': 'подозреваемого'}
вынес {'subj': 'суд', 'obj': 'приговор'}
закрасили {'subj': 'Чиновники', 'obj': 'мемориал'}
продлил {'subj': 'Клуб', 'obj': 'контракт'}


#### Задание
Измените код выше так, чтобы учитывались:
    1. Однородные члены предложения 
        * (парк, площадка), (Германия, Щвейцария)
    2. Сложные сказуемые 
        * (начнет продавать), (запретил провозить)
    3. Непрямые объекты
        * (едет, Польшу), (спел, скандале)